# CW-Husky ADC Sampling Phase Exploration

With CW-Husky, the relative phase between the target clock and the ADC sampling clock can be adjusted via `scope.clock.adc_phase`.

`scope.clock.adc_phase` has a limited range which depends on the target clock frequency; the first and main purpose of this notebook is to teach the mechanics of phase adjustments.

We use Husky's built-in logic analyzer to do so.

The phase relationship between the target and ADC clocks depends on whether:
1. Husky is generating the target clock;
2. The target is generating its own clock, and Husky uses this clock to generate a synchronous ADC sampling clock.

(There is a third scenario, where Husky does not have access to the target clock and is therefore sampling asynchronously; in this scenario `scope.clock.adc_phase` is irrelevant.)

The second purpose of this notebook is to explain the changes to clock generation that were done to resolve issues [490](https://github.com/newaetech/chipwhisperer/issues/490), [499](https://github.com/newaetech/chipwhisperer/issues/499), and [501](https://github.com/newaetech/chipwhisperer/issues/501). The intent is to help you understand how these changes affect the clock phase, and whether and how you can make updates to your existing ChipWhisperer capture scripts. These issues were fixed on the ChipWhisperer develop branch in November 2024.

## Part 1: Husky Generates the Target Clock.

No target is required for this part.

In [ ]:
SCOPE="OPENADC"
PLATFORM="CWHUSKY"

In [ ]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()

# avoid warnings when we slightly exceed the max clock spec:
scope.trace.clock._warning_frequency = 255e6

In [ ]:
EXTCLK = False
target = None
scope.clock.clkgen_src = 'system'

scope.LA.enabled = True
scope.LA.clk_source = 'pll'
scope.LA.clkgen_enabled = False
scope.LA.oversampling_factor = 1
scope.LA.clkgen_enabled = True
scope.LA.capture_group = 'CW 20-pin'
scope.LA.capture_depth = 500
assert scope.LA.locked

We define a convenience function for setting the clock parameters. It also sets the `scope.LA` sampling rate accordingly for maximum precision.

In [ ]:
def set_freq(freq, adc_mul, adc_phase):
    scope.clock.adc_mul = adc_mul
    scope.clock.clkgen_freq = freq
    scope.clock.adc_phase = adc_phase

    oversamp = int(250e6//freq)
    scope.LA.clkgen_enabled = False
    scope.LA.oversampling_factor = oversamp
    scope.LA.clkgen_enabled = True
    assert scope.LA.locked

Let's start with a simple example: our standard 7.37 MHz clock, `scope.clock.adc_mul = 1`, and `scope.clock.adc_phase = 0`:

In [ ]:
set_freq(7.37e6, 1, 0)

These methods capture the target and ADC clocks using `scope.LA`; we'll use them many times throughout this notebook:

In [ ]:
def find0to1trans(data):
    pattern = [0,1]
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

def get_clocks():
    done = False
    count = 0
    samples = scope.LA.oversampling_factor * 4
    while not done and count < 30:
        scope.LA.arm()
        scope.LA.trigger_now()
        raw = scope.LA.read_capture_data()
        adcclock = scope.LA.extract(raw, 8)
        if EXTCLK:
            refclock = scope.LA.extract(raw, 4)
        else:
            refclock = scope.LA.extract(raw, 5)
        edges = find0to1trans(refclock)
        if len(edges) > 1:
            ref_edge = edges[1]
        else:
            ref_edge = edges[0]    
        try:
            #adc_ref_delta = find0to1trans(adcclock[ref_edge:])[0]
            adc_edges = find0to1trans(adcclock)
            adc_ref_delta = abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge)
            done = True
        except:
            # sometimes (rarely) the ADC clock comes back all zeros; could be an issue with the PLL or with the LA?
            if all(c == 0 for c in adcclock):
                adcclock = 'all zeros'
            print('could not find delta; ref_edge=%3d, lock status=%s; adcclock=%s; trying again' % (ref_edge, scope.clock.pll.pll_locked, adcclock))
            assert scope.LA.locked
            assert scope.clock.pll.pll_locked
            time.sleep(0.5)
            count += 1
    return adc_ref_delta, adcclock[edges[0]:edges[0]+samples], refclock[edges[0]:edges[0]+samples]

Let's see what we get:

In [ ]:
delta, adcclock, refclock = get_clocks()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

In [ ]:
samples = len(adcclock)
xrange = list(range(samples))
p = figure(width=1800, height=400)

refline = p.line(xrange, refclock, line_color='blue')
adcline = p.line(xrange, adcclock-2, line_color='green')
difline = p.line(xrange, abs(refclock-adcclock)-4, line_color='red')

legend = Legend(items=[
    LegendItem(label='target clock', renderers=[refline]),
    LegendItem(label='ADC clock', renderers=[adcline]),
    LegendItem(label='difference between clocks', renderers=[difline]),
])
p.add_layout(legend)
show(p)

The clocks should be coincident or nearly so. Now let's play with `scope.clock.adc_phase`.

The [CDCI6214](https://www.ti.com/product/CDCI6214) is the PLL that's generating the clocks for CW-Husky, and it allows its output clocks to be delayed by relatively small sub-period steps; by delaying either one or both of the target and ADC clocks, we can obtain a different phase relationship between these two clocks.

The step size depends on the clock frequency (not in a straightforward/linear way: it depends on the PLL's settings). Query this property to know the current step size, in picoseconds:

In [ ]:
scope.clock.adc_phase_step_size

Now the phase itself is set via `scope.clock.adc_phase`, which is expressed in percentage of the target clock period.

Let's measure and plot a few different values. We'll shift the ADC clock +/- 25% of the clock period:

In [ ]:
set_freq(7.37e6, 1, +25)
delta, adcclock_p25, refclock_p10 = get_clocks()

set_freq(7.37e6, 1, -25)
delta, adcclock_m25, refclock_m10 = get_clocks()

In [ ]:
samples = len(adcclock)
xrange = list(range(samples))
p = figure(width=1800, height=400)

refline = p.line(xrange, refclock, line_color='blue')
adcline = p.line(xrange, adcclock-2, line_color='green')
adcline_p25 = p.line(xrange, adcclock_p25-4, line_color='orange')
adcline_m25 = p.line(xrange, adcclock_m25-6, line_color='red')


legend = Legend(items=[
    LegendItem(label='target clock', renderers=[refline]),
    LegendItem(label='ADC clock', renderers=[adcline]),
    LegendItem(label='ADC clock, scope.clock.adc_phase = +25', renderers=[adcline_p25]),
    LegendItem(label='ADC clock, scope.clock.adc_phase = -25', renderers=[adcline_m25]),
])
p.add_layout(legend)
show(p)

`scope.clock.adc_phase` has a limited range: the highest it can go depends on the target clock frequency (again, not in a straightforward way). The relationship between max phase and clock frequency is not a simple one, so we provide the `scope.clock.pll.max_phase_percent` property:

In [ ]:
scope.clock.pll.max_phase_percent

If you try to set `scope.clock.adc_phase` above this, you'll get an error.

We've used a fixed clock and `adc_mul` values to show the basics here, but you're not limited to these values. Play around with them as you like, keeping in mind that we set the maximum `scope.LA` sampling rate to 250 MS/s; the faster the clocks, the lower the granularity.

In [ ]:
set_freq(8e6, 2, +15)
delta, adcclock, refclock = get_clocks()

In [ ]:
set_freq(15e6, 1, 0)
delta, adcclock, refclock = get_clocks()

In [ ]:
samples = len(adcclock)
xrange = list(range(samples))
p = figure(width=1800, height=400)

refline = p.line(xrange, refclock, line_color='blue')
adcline = p.line(xrange, adcclock-2, line_color='green')

legend = Legend(items=[
    LegendItem(label='target clock', renderers=[refline]),
    LegendItem(label='ADC clock', renderers=[adcline]),
])

p.add_layout(legend)
show(p)

### Clock phase prior to issue [490](https://github.com/newaetech/chipwhisperer/issues/490) being fixed:

*In most situations are for most users, you don't need to worry about this.*

*In particular, if you never set `scope.clock.adc_phase != 0` and always used `scope.clock.clkgen_src = system`, you are not affected.*

When Husky generates the target clock, the root cause behind issue [490](https://github.com/newaetech/chipwhisperer/issues/490) is that when the user changed `scope.clock.adc_mul`, the PLL driver code would try very hard to find PLL parameters which would not affect (i.e. momentarily drop) the target clock.

Though well-intentioned, this had the side-effect that going from a set of clock settings "A" to a set of clock settings "B" could yield different PLL settings than going from "A" to some other settings "X" before ending on "B".

In other words, an innocuous change to a capture script -- adding or remove "X" in the example above -- could result in different PLL settings, even though the user is requesting a final set of clock settings "B" in both cases.

Why is this a problem? We saw above that the dimension of the `adc_phase` step depends on PLL settings. Prior to November 2024, `scope.clock.adc_phase` was specified as a dimensionless integer, whose actual meaning (in terms of target/ADC clock phase) depends on the PLL parameters. If you had `scope.clock.adc_phase = 0`, then you can ignore all this because 0 is always 0, in both cases. But if you used a non-zero `adc_phase`, your actual phase may have been inconsistent.

To avoid this situation, `scope.clock.adc_phase` is now expressed in percentage of the target clock period. 

The phase can still be specified as a raw integer in the range -31 to +31 if you wish to attempt to replicate a previously used phase, via `scope.clock.adc_phase_raw`. (Previously, the phase was a integer in the range -255 to +255; simply scale from [-255,255] to [-31,31] linearly.)

You can use this notebook with an older ChipWhisperer release or commit to visualize what phase you had, and find the best parameters for you desired phase for the current ChipWhisperer code.

# Part 2: The Target Generates its Own Clock

Here we'll use a CW305 to provide a clock to Husky. You can use a different target to provide the clock; simply modify the `set_target_freq()` method below to what's required to set your target's clock frequency.

In [ ]:
EXTCLK = True
scope.clock.clkgen_freq = 10e6
scope.clock.clkgen_src = 'extclk'
scope.LA.clk_source = 'target'

In [ ]:
def set_target_freq(freq):
    # this is for setting the CW305's clock; modify as needed for different targets
    global target
    if target is None:
        target = cw.target(scope, cw.targets.CW305, fpga_id='100t')
        target.pll.pll_enable_set(True)
        target.pll.pll_outenable_set(False, 0)
        target.pll.pll_outenable_set(True, 1)
        target.pll.pll_outenable_set(False, 2)
        target.pll.pll_outfreq_set(10e6, 1)
    target.pll.pll_outfreq_set(freq, 1)

We'll modify `set_freq()` so that it also sets the target's frequency:

In [ ]:
def set_freq(freq, adc_mul, adc_phase):
    set_target_freq(freq)
    scope.clock.adc_mul = adc_mul
    scope.clock.clkgen_freq = freq
    scope.clock.adc_phase = adc_phase

    oversamp = int(250e6//freq)
    scope.LA.clkgen_enabled = False
    scope.LA.oversampling_factor = oversamp
    scope.LA.clkgen_enabled = True
    assert scope.LA.locked

In [ ]:
set_freq(10e6, 1, 0)

Let's collect measurements and plot:

In [ ]:
delta, adcclock, refclock = get_clocks()

samples = len(adcclock)
xrange = list(range(samples))
p = figure(width=1800, height=400)

refline = p.line(xrange, refclock, line_color='blue')
adcline = p.line(xrange, adcclock-2, line_color='green')

legend = Legend(items=[
    LegendItem(label='target clock', renderers=[refline]),
    LegendItem(label='ADC clock', renderers=[adcline]),
])
p.add_layout(legend)
show(p)

Why the large phase difference with `scope.clock.adc_phase = 0`? The clock distribution network in this scenario, illustrated below, is far from simple:
1. the target generates a clock, which is sent to Husky on the HS1 pin;
2. the Husky FPGA takes this clock and passes it onto the CDCI6214 PLL, for it to use as a reference;
3. the PLL generates the ADC sampling clock from this reference and sends it to the ADC;
4. the ADC sends a copy of this clock back to the Husky FPGA.

The actual target and ADC clocks are highlighted green. The versions of these clocks that are sampled by `scope.LA` are highlighted yellow.

Each black box in the diagram is a discrete chip, with associated pad delays and routing delays.

That's a lot of clock routing, across three different chips. Husky's `scope.LA` isn't measuring these clock where they are used; they're measured somewhere further along.

<img src="img/husky_clocks_phase.png" width="30%"/>

The phase relationship between the target and ADC clocks, **where they are measured by `scope.LA`**, when `scope.clock.adc_phase = 0` is as follows: the rising edge of the ADC clock follows approximately 10ns after the *falling* edge of the target clock.

The ~10ns offset is independent of the target (or ADC) clock frequency. To verify, we measure the clocks when HS1 is swept from 10 MHz to 30 MHz:

In [ ]:
from tqdm.notebook import tnrange, tqdm
import numpy as np
import time
import random

adcs = []
refs = []

for freq in tnrange(int(10e6), int(30e6), int(2e5)):
    set_freq(freq, 1, 0)
    delta, adc, ref = get_clocks()
    adcs.append(adc)
    refs.append(ref)

In [ ]:
def update_plot(f):
    start = find1to0trans(refs[f])[0]
    S1.data_source.data['y'] = refs[f][start-1:start-1+samples] + 2
    S2.data_source.data['y'] = adcs[f][start-1:start-1+samples] + 0  
    push_notebook()

def find1to0trans(data):
    pattern = [1,0]
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

p = 0

S = figure(width=1800)

samples = int(len(adcs[-1])*3/4)+1
xrange = list(range(samples))

start = find1to0trans(refs[p])[0]

S1 = S.line(xrange, refs[p][start-1:start-1+samples] + 2, line_color='black')
S2 = S.line(xrange, adcs[p][start-1:start-1+samples] + 0, line_color='blue')

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, f=(0, len(adcs)-1))

### Clock phase prior to issues [499](https://github.com/newaetech/chipwhisperer/issues/499) and [501](https://github.com/newaetech/chipwhisperer/issues/501) being fixed:

*In most situations are for most users, you don't need to worry about this.*

*None of our notebooks which use `scope.clock.clkgen_src = 'extclk'` (AES and ECC FPGA target notebooks) required modifications.*


#### Issue [499](https://github.com/newaetech/chipwhisperer/issues/499):
It was previously possible that Husky would generate an ADC clock that was close to, but not exactly, an `scope.clock.adc_mul` multiple of the target clock.

Obviously this would result in a constantly changing phase between the ADC and target clocks.

Follow the issue link above to learn how to find out whether your particular configuration of clocks ran into this issue.

#### Issue [501](https://github.com/newaetech/chipwhisperer/issues/501):
The phase offset would have a frequency-dependent component (i.e. in the interactive plot above, the ADC clock rising edge would be moving around).

You can use this notebook with an older ChipWhisperer release or commit to visualize what phase you had, and find the `scope.clock.adc_phase` which most closely matches this with the current ChipWhisperer code.

Among the `scope.clock.adc_phase` issues, this is the one most likely to impact you (**if** you use `scope.clock.clkgen_src = 'extclk'`). The impact would be largest if you use `scope.clock.adc_mul = 1`, and used a clock frequency that results in a significantly different phase post-fix.